In [1]:
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [2]:
class MLP(Model):
    def __init__(self, hidden_dim, output_dim):
        super().__init__()
        self.l1 = Dense(hidden_dim,activation='sigmoid')
        self.l2 = Dense(output_dim,activation='sigmoid')
    
    def call(self,x):
        h = self.l1(x)
        y = self.l2(h)
        return y

In [3]:
np.random.seed(123)
tf.random.set_seed(123)
criterion = losses.BinaryCrossentropy()
optimizer = optimizers.SGD(learning_rate=0.1)

N = 300
x, t = datasets.make_moons(N,noise=0.3)
t = t.reshape(N,1)
x_train, x_test, t_train, t_test = train_test_split(x, t,test_size=0.2)

model = MLP(3,1)

def compute_loss(t,y):
    return criterion(t,y)

def train_step(x,t):
    with tf.GradientTape() as tape:
        preds = model(x)
        loss = compute_loss(t,preds)
    grads = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    return loss

epochs = 100
batch_size = 10
n_batches = x_train.shape[0] // batch_size

for epoch in range(epochs):
    train_loss = 0
    x_,t_ = shuffle(x_train,t_train)

    for batch in range(n_batches):
        start = batch * batch_size
        end = start + batch_size
        loss = train_step(x_[start:end], t_[start:end])
        train_loss += loss.numpy()
    
    print([epoch + 1, train_loss])

[1, 18.153739392757416]
[2, 17.146430134773254]
[3, 16.422325909137726]
[4, 15.800965785980225]
[5, 15.259121835231781]
[6, 14.734732508659363]
[7, 14.255744636058807]
[8, 13.820853531360626]
[9, 13.412509262561798]
[10, 13.037162274122238]
[11, 12.677453428506851]
[12, 12.34906181693077]
[13, 12.053036451339722]
[14, 11.770999103784561]
[15, 11.523473352193832]
[16, 11.319578349590302]
[17, 11.120020121335983]
[18, 10.942359983921051]
[19, 10.779083460569382]
[20, 10.65008920431137]
[21, 10.54791758954525]
[22, 10.431716367602348]
[23, 10.317040264606476]
[24, 10.24480353295803]
[25, 10.15718786418438]
[26, 10.112668424844742]
[27, 10.05280590057373]
[28, 9.970908209681511]
[29, 9.924373105168343]
[30, 9.893476843833923]
[31, 9.850363597273827]
[32, 9.816470548510551]
[33, 9.768211424350739]
[34, 9.74425257742405]
[35, 9.719978675246239]
[36, 9.693762242794037]
[37, 9.679912835359573]
[38, 9.669430986046791]
[39, 9.645051956176758]
[40, 9.606713145971298]
[41, 9.612317502498627]
[42, 

In [4]:
test_loss = metrics.Mean()
test_acc = metrics.BinaryAccuracy()

def test_step(x,t):
    preds = model(x)
    loss = compute_loss(t,preds)
    test_loss(loss)
    test_acc(t,preds)
    return preds

test_step(x_test,t_test)
print([test_loss.result().numpy(),test_acc.result().numpy()])

[0.31024387, 0.9]
